# Assignment 3

## *Generating useful features for further analysis on Amazon reviews*
<h2>Introduction</h2>
<p><strong>Business Context.</strong> You are a business consultant with new clients that are interested in analyzing reviews of their products on Amazon (as opposed to Yelp). They want to answer business questions like: "What are the most important factors driving negative reviews?", "Have there been any large changes to customer satisfaction/reviews over time?", etc.</p>
<p><strong>Business Problem.</strong> Your main task is to <strong>explore the given data and use the results of your investigation to engineer relevant features that could facilitate subsequent analysis and model-building</strong>.</p>
<p><strong>Analytical Context.</strong> The dataset provided is a large body of reviews related to movies and television left on Amazon between 1996 and 2014. When exploring our dataset, we will quickly encounter a familiar problem we discussed in the previous case: the word "good" is one of the most important words in both positive <em>and</em> negative reviews. Thus, we must develop methods to put "good" in the appropriate context.</p>

<h2>Loading the data</h2>
<p>We use a dataset of around 37,000 video reviews from Amazon Instant Video and 1,700,000 movie and TV reviews, all obtained from the website: http://jmcauley.ucsd.edu/data/amazon/. Note that there are much larger datasets available at the same site. We can expect better and more consistent results on larger datasets (such as book reviews). Note that these datasets are compressed (gzipped), and they are in <a href="https://en.wikipedia.org/wiki/JSON">JSON</a> format, with each line representing a review and each line being its own JSON object.</p>
<p>We begin by loading the dataset below:</p>

In [ ]:
%matplotlib inline

In [ ]:
%%time
import gzip
import json
import string
import re

import nltk # imports the natural language toolkit
import pandas as pd
import plotly

from collections import Counter
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('punkt')


# we can tell pandas that our file is in gzip format and it will handle the decompression for us
# we also use `lines=True` to indicate that each line of the file is its own JSON object
instant_video = pd.read_json("reviews_Amazon_Instant_Video_5.json.gz", lines=True, compression='gzip')


# ----------
# The Movies and TV file is very big. If you have problems loading it, you can load only the first 
# 100,000 reviews by using 'chunksize' (uncomment the line with 'chunksize' and comment out the line 
# after that which loads the entire file into `movies_tv`). All of the analysis can be 
# done in the same way using only the subset of reviews but some of the results might be different from the examples.
# ----------
# movies_tv = next(pd.read_json("reviews_Movies_and_TV_5.json.gz", lines=True, compression='gzip', chunksize=10000))
movies_tv = pd.read_json("reviews_Movies_and_TV_5.json.gz", lines=True, compression='gzip')

<h2>Examining the data</h2>
<p>We take a look at the first 5 rows of each dataset to see what attributes are available. These are</p>
<ul>
<li><strong>reviewerID:</strong> A unique ID to identify the author of the review.</li>
<li><strong>asin:</strong> The <a href="https://www.amazon.com/gp/seller/asin-upc-isbn-info.html">"Amazon Standard Identification Number"</a> which provides more information about the exact product and version.</li>
<li><strong>reviewerName:</strong> The username chosen by the reviewer.</li>
<li><strong>helpful:</strong> A record of how many users indicated that the review was helpful/not helpful.</li>
<li><strong>reviewText:</strong> The full text of the review.</li>
<li><strong>overall:</strong> The overall rating (1-5) left by the reviewer.</li>
<li><strong>summary:</strong> A short version of the review, used as the title.</li>
<li><strong>unixReviewTime:</strong> The date that the review was created, in <a href="https://en.wikipedia.org/wiki/Unix_time">Unix Epoch</a> format.</li>
<li><strong>reviewTime:</strong> A human readable date giving the day, month, and year.
</font></li>
</ul>

In [ ]:
print(len(instant_video))
print(instant_video.head(5))

In [ ]:
print(len(movies_tv))
print(movies_tv.head(5))

<p>We notice that <code>movies_tv</code> is extremely long with nearly 2 million reviews, and several columns seem uninteresting or hard to work with (e.g. <code>reviewerID</code>, <code>asin</code>, <code>reviewername</code>, <code>reviewtime</code>). We drop some information to make some of our later analysis more efficient. We also add a datetime column with Python datetime objects to more easily summarize the data:</p>

In [ ]:
%%time
movies_tv['datetime'] = pd.to_datetime(movies_tv['reviewTime'], format="%m %d, %Y")
instant_video['datetime'] = pd.to_datetime(instant_video['reviewTime'], format="%m %d, %Y")

In [ ]:
movies_tv = movies_tv.drop(columns = ['reviewerID', 'asin', 'reviewerName', 'reviewTime'])
instant_video = instant_video.drop(columns = ['reviewerID', 'asin', 'reviewerName', 'reviewTime'])

movies_tv.head(5)

<h3>Exercise 1:</h3>
<h4>1.1</h4>
<p>Plot histograms of all numeric quantities. Do you notice anything interesting about them?</p>

**Answer.**

-------

<h4>1.2</h4>
<p>How do average ratings change over time? Plot the average rating for each year and note any trends.</p>

**Answer.**

-------

<h4>1.3</h4>
<p>Look at the average length of the review by year. Do you notice any trends?</p>

**Answer.**

-------

<h3>Exercise 2:</h3>
<h4>2.1</h4>
<p>Find the ten most frequently occuring non-stop words acrooss: (i) all reviews, (ii) positive reviews, (iii) negative reviews. Do the results surprise you? Why or why not?</p>

**Answer.**

-------

<h4>2.2</h4>
<p>Find words that are indicative of bad reviews. That is, words that appear often in bad reviews and <em>not</em> in good reviews. What are these words and are they surprising?</p>

**Answer.**

-------

<h3>Exercise 3:</h3>
<p>Manually inspect the first 10 negative reviews containing the word "good". What do you notice? How does this suggest we ought to proceed next?</p>

**Answer.**

-------

<h3>Exercise 4:</h3>
<p>Go through the list of bad reviews containing the word "good" that we found in the last question. For each review, extract the following:</p>
<ol>
<li>The first word after "good"</li>
<li>The first word after "good" that is a noun or cardinal</li>
<li>The last word before "good" that is a noun or cardinal</li>
</ol>

**Answer.**

-------

<h3>Exercise 5:</h3>
<p>We have seen that individual words are not always very informative. Look for the most informative bigrams and trigrams, in both positive and negative reviews. Show the most informative bigrams and trigrams and give a brief analysis of the n-grams you identified.</p>

**Answer.**

-------

<h3>Exercise 6:</h3>
<p>Throughout the above search for informative words, we have seen that unigrams are not enough, but important words (such as "good") are not always next to the informative words that they describe. Devise a method to extract these informative words. Provide a brief description of how you will extract the informative words.</p>

**Answer.**

-------

<h3>Exercise 7:</h3>
<p>Write a function(s) that transforms a sentence into a new text list by iteratively pairing each adjective in the sentence with the next noun that follows it in the sentence. For example, the text "That was a good, long movie" should return <code>["good movie", "long movie"]</code>.</p>

**Answer.**

-------